# Transfer learning

In this lab we will make use of pretrained models in order to boost performance on smaller datasets. For this experiment, we will be working with an AlexNet model pretrained on the Imagenet dataset in order to get a good accuracy score on the Caltech 101 dataset.

### Prerequisites

1. In order to perform the experiments, please download in advance the Caltech 101 dataset from https://drive.google.com/file/d/137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp/view
2. In the working directory please create a folder named 'dataset' and a subfolder named 'caltech101' within it. Extract the dataset in the subfolder. The overall folder structure should look as follows: dataset/caltech101/101_ObjectCategories.
3. Install the torchvision module using 'conda install torchvision' if you have not done so already.

In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torchvision
import warnings

warnings.filterwarnings('ignore')

Firstly, we will load the AlexNet model architecture using torchvision. All available models with their respective parameters can be found at: https://pytorch.org/vision/stable/models.html

In [ ]:
model = torchvision.models.alexnet(pretrained = True)

In the first run we will just load the model architecture, without the pretrained weights. We can visualize the model architecture as follows:

In [ ]:
model

Next, we will load the Caltech 101 dataset and apply the neccesary transformations on it. Afterwards, we will split the dataset into train, validation and test.

In this block of code, define the dataloaders for train, validation and test and try to iterate through the data. What happens? Try to fix the problem using a lambda transform: https://pytorch.org/vision/stable/transforms.html#generic-transforms

In [ ]:
dataset = torchvision.datasets.Caltech101(
    './dataset',
    transform = torchvision.transforms.Compose([
        torchvision.transforms.PILToTensor(),
        torchvision.transforms.ConvertImageDtype(torch.float),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.Lambda(
            lambda img: img if img.shape[0] == 3 else img.repeat(3, 1, 1)
        )
        # add a lambda transform in order to fix the problem
    ])
)

n_samples = len(dataset)
n_train_samples = int(.8 * n_samples)
n_val_samples = int(.1 * n_samples)
n_test_samples = n_samples - n_train_samples - n_val_samples

train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [
    n_train_samples, n_val_samples, n_test_samples
])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size = 128, shuffle = True, num_workers = 2)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size = 128, shuffle = True, num_workers = 2)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size = 128, shuffle = True, num_workers = 2)

With the dataset ready, it is now time to adapt the model architecture in order to fit our needs. Define a new classifier for the AlexNet model having the same structure, changing only the number of output neurons to 101.

In [ ]:
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=9216, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=4096, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(in_features=4096, out_features=1000, bias=True),
)
# model.classifier[6] = torch.nn.Linear(in_features = 4096, out_features = 101)

### Training the model

Define an Adam optimizer with a learining rate of 1e-4 and a cross entropy loss. Afterwards, train the model for 2 epochs. Note the results

In [ ]:
# train the model for 2 epochs using a cross-entropy loss and an Adam optimizer with a lr of 1e-4

device = torch.device('cpu')
loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

model.to(device)

# freezing pretrained layers
for param in model.features.parameters():
    param.requires_grad = True

best_val_acc = 0
for epoch_n in range(2):
    model.train()
    for batch in tqdm(train_dl):
        model.zero_grad()

        input, target = batch 
        input = input.float().to(device)
        target = target.to(device)

        output = model(input)
        loss = loss_f(output, target)

        loss.backward()
        optimizer.step()

    # validate model
    model.eval()
    accs = []
    for batch in val_dl:
        input, target = batch 
        input = input.float().to(device)
        target = target.to(device)

        with torch.no_grad():
            output = model(input)

        predictions = output.argmax(1)
        acc = (predictions == target).float().mean().detach().cpu().numpy()
        accs.append(acc)
    val_acc = np.mean(accs)
    print(val_acc)


## Experiments:

1. Rerun training (restart kernel and run all cells) but this time, when loading the model in the first block of code, specify 'pretrained = True' in order to make use of the weights pretrained on Imagenet.
2. Rerun the code using the pretrained model but this time use a learning rate of 1e-3. What happens?
3. Rerun using the pretrained model and a lr of 1e-4 but this time only change the last layer in the model instead of the entire classifier.
3. Rerun the code using the pretrained model and a lr of 1e-4. This time, freeze the pretrained layers and only update the new layers for the first epochs. Afterwards, proceed to update the entire model. You can freeze parameters by specifying 'requires_grad = False'.
4. Rerun experiment 3 but gradually unfreeze layers instead of unfreezeing the entire model at once.